In [1]:
from sqlalchemy import create_engine
import pandas as pd
import random 

In [3]:
#Define the connection string
cnxn_string =("postgresql+psycopg2://{username}:{pswd}"
              "@{host}:{port}/{database}")

engine = create_engine(cnxn_string.format(
    username="postgres",
    pswd="*******",
    host="localhost",
    port=5432,
    database="dvdrental"))

# Quick Case 1:
* Say we wanted to explore customers first order value ($), How much they have spent over 14 days since that initial order and the total amount they spent over all the time they've been a customer

In [8]:
query = """ 
        WITH bt as(
        SELECT t.* 
        FROM
        (
            SELECT p.customer_id, p.payment_date, row_number() OVER (PARTITION BY p.customer_id ORDER BY p.payment_date) as row_number
            FROM payment p)t
        WHERE t.row_number = 1 
        )

        SELECT  bt.customer_id, bt.payment_date, 

                (
                    SELECT SUM(p2.amount) FROM payment p2 WHERE p2.customer_id = bt.customer_id 
                                                        AND p2.payment_date BETWEEN bt.payment_date AND bt.payment_date + INTERVAL '7 days'
                ) AS first7_sales,

                (
                    SELECT SUM(p2.amount) FROM payment p2 WHERE p2.customer_id = bt.customer_id 
                                                        AND p2.payment_date BETWEEN bt.payment_date AND bt.payment_date + INTERVAL '14 days'
                ) AS first14_sales,

                (
                    SELECT SUM(p2.amount) FROM payment p2 WHERE p2.customer_id = bt.customer_id 
                ) AS LifeTimeValue

        FROM bt
"""

result = pd.read_sql_query(query,engine)
result

,customer_id,payment_date,first7_sales,first14_sales,lifetimevalue
0,1,2007-02-14 23:22:38.996577,31.93,31.93,114.70
1,2,2007-02-17 19:23:24.996577,2.99,20.94,123.74
2,3,2007-02-16 00:02:31.996577,25.96,31.95,130.76
3,4,2007-02-15 07:59:54.996577,16.94,16.94,81.78
4,5,2007-02-15 20:31:40.996577,19.95,25.93,134.65
...,...,...,...,...,...
594,595,2007-02-14 22:16:01.996577,7.98,7.98,110.71
595,596,2007-02-16 07:26:44.996577,3.98,17.94,73.78
596,597,2007-02-18 13:28:05.996577,7.97,24.93,87.77
597,598,2007-02-20 08:38:55.996577,2.99,3.98,83.78


* Now we can take those results into Tableau, Excel, PBI etc. and explore it with further analysis.
* For example we can see that the customer with customer id = 1, did not purchase anything in the time interval of 2 weeks since joining
* Feel free to find your own analysis ideas and take it from there


# Quick Case 2:
* We want to calculate the CPA (Cost Per Action) and Profitability Analysis
* Using the DB dvdrental, generating synthetic user data and source data


In [ ]:
# Step 1 - Create two new table to populate later

create_tables = """
    CREATE TABLE IF NOT EXISTS customer_sources(
        customer_id integer REFERENCES customer(customer_id) ON DELETE RESTRICT,
        traffic_source text,
        PRIMARY KEY(customer_id)
    );

    CREATE TABLE IF NOT EXISTS source_spend_all(
        spend_source text,
        spend integer,
        visits integer
    );
"""
pd.read_sql_query(create_tables,engine)



* customer_sources will be the table with customer_id, traffic source
* source_spend_all will be the table with source name (traffic source), spend ($), visits

In [ ]:
# Step 2 - Generate some data for the tables

#Create source
sources = ['google / cpc', 'google / organic', 'bing / cpc', 'moviereviews / display', 'direct / none', 'yelp / referral']
#Select randomly from list random.choices(list, weights, k=number of items)
traffic = random.choices(sources, weights = [20,10,3,5,15,8], k =599)
#Create enumarated list
ids = [(idx+1,x) for idx,x in enumerate(traffic)]
#ids -- uncomment to view

#Create dataframe
df = pd.DataFrame.from_records(ids)
df.columns = ['user_id', 'source_medium']
#df.head() -- uncomment to view 

#Create the data for the 2nd table
data = [['direct / none',0,755],['google / organic',750,455],['google / cpc',1606,955], ['bing / cpc',133,45],['moviereviews / display',2886,1200],['yelp / referral',0,99]]
#Create dataframe
source_spend_all_df = pd.DataFrame(data)
#source_spend_all_df.head() -- uncomment to view 

#Export to csv
df.to_csv('customer_sources.csv', sep=',', index=False, header=False)
source_spend_all_df.to_csv('source_spend_all.csv', sep=',', index=False, header=False)

Next step is to populate this info in PgAdmin4

In [9]:
#Step 3 - Check everything was done correctly

customer_sources_query = "SELECT * FROM customer_sources"
source_spend_all_query = "SELECT * FROM source_spend_all"


In [10]:
customer_sources_result = pd.read_sql_query(customer_sources_query,engine)
customer_sources_result

,customer_id,traffic_source
0,1,direct / none
1,2,direct / none
2,3,direct / none
3,4,google / organic
4,5,google / organic
...,...,...
594,595,google / cpc
595,596,google / organic
596,597,direct / none
597,598,direct / none


In [11]:
source_spend_all_result = pd.read_sql_query(source_spend_all_query,engine)
source_spend_all_result

,spend_source,spend,visits
0,direct / none,0,755
1,google / organic,750,455
2,google / cpc,1606,955
3,bing / cpc,133,45
4,moviereviews / display,2886,1200
5,yelp / referral,0,99


Good, all the data was imported properly

In [14]:
#STEP 4 - Let's create a DF that will be easy to analyze in our BI tool

query = """
    SELECT t.spend_source, max(t.spend)::money as spend, count(*) as count_customers, 
        (max(t.spend)/count(*))::money as CPA,
        SUM(t.LTV)::money as total_customer_spend,
        (SUM(t.LTV)/count(*))::money AS RPA
    FROM(
        SELECT ssa.*, cs.*, (
            SELECT SUM(p.amount) FROM payment p WHERE cs.customer_id = p.customer_id
        ) as LTV
        FROM source_spend_all ssa 
        JOIN customer_sources cs ON cs.traffic_source=ssa.spend_source)t
    GROUP BY 1
    ORDER BY 2 DESC
"""

result = pd.read_sql_query(query,engine)
result

,spend_source,spend,count_customers,cpa,total_customer_spend,rpa
0,moviereviews / display,"$2,886.00",43,$67.00,"$4,314.65",$100.34
1,google / cpc,"$1,606.00",195,$8.00,"$20,675.23",$106.03
2,google / organic,$750.00,96,$7.00,"$9,621.70",$100.23
3,bing / cpc,$133.00,38,$3.00,"$3,892.74",$102.44
4,direct / none,$0.00,165,$0.00,"$16,531.48",$100.19
5,yelp / referral,$0.00,62,$0.00,"$6,276.24",$101.23


* Now we can take those results into Tableau, Excel, PBI etc. and explore it with further analysis.
* For example we can see that the moviereviews site with display adds costs way too much, and let's assume we 
need to pay for movie rights etc. in that case we have a source that is causing us to bleed money.
* Feel free to find your own analysis ideas and take it from there

# END